In [1]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm
from collections import defaultdict

# =====================================================
# ✅ 版本控制变量 —— 只需修改这一行
# =====================================================
VERSION = "0.5"
SCORE_THRESHOLD = 0.7  # 可灵活修改置信度阈值

# =====================================================
# 路径与模型配置
# =====================================================
BASE_DIR = "/opt/data/private/BlackBox"
SAVE_DIR = f"{BASE_DIR}/save-{VERSION}/attack/detection"
ANNOT_PATH = f"{BASE_DIR}/data/coco-patch-{VERSION}/annotations/instances_val2017.json"

MODEL_DIRS = {
    "detr": f"{SAVE_DIR}/detr/patch/",
    "deformable-detr": f"{SAVE_DIR}/deformable-detr/patch/",
    "sparse-detr": f"{SAVE_DIR}/sparse-detr/patch/",
    "anchor-detr": f"{SAVE_DIR}/anchor-detr/patch/",
    "dn-detr": f"{SAVE_DIR}/dn-detr/patch/",
}

# =====================================================
# 工具函数
# =====================================================
def is_patch_inside_box(box, patch):
    """
    判断 patch 是否完全在检测框 box 内部
    box:   [x1, y1, x2, y2]
    patch: [px1, py1, px2, py2]
    满足条件: 检测框完全包住 patch
    """
    x1, y1, x2, y2 = box
    px1, py1, px2, py2 = patch
    return x1 <= px1 and y1 <= py1 and x2 >= px2 and y2 >= py2


def load_patch_mapping(annotations_path):
    """加载 image_id → patch_regions 映射"""
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    patch_mapping = {}
    for img in data["images"]:
        img_id = img["id"]
        patches = img.get("patch_regions", [])
        patch_mapping[img_id] = patches
    return patch_mapping


def filter_json(input_path, patch_mapping, score_thr=0.7):
    """筛选: person + score≥thr + 检测框完全包含patch"""
    if not os.path.exists(input_path):
        print(f"⚠️ 文件不存在: {input_path}")
        return None, None

    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if not isinstance(data, list):
        print(f"⚠️ 文件格式异常(非list): {input_path}")
        return None, None

    filtered = []
    for det in data:
        if det.get("category_id") != 1 or det.get("score", 0) < score_thr:
            continue

        img_id = det.get("image_id")
        if img_id not in patch_mapping:
            continue

        bbox = det.get("bbox", [])
        if len(bbox) != 4:
            continue

        patches = patch_mapping[img_id]
        # ✅ 保留：检测框完全包住任意一个 patch
        keep = any(is_patch_inside_box(bbox, patch) for patch in patches)

        if keep:
            filtered.append(det)

    grouped = defaultdict(list)
    for det in filtered:
        grouped[det["image_id"]].append(det)

    return filtered, grouped


def save_filtered(filtered, output_path):
    """保存筛选结果"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(filtered, f, indent=2, ensure_ascii=False)
    print(f"✅ 已保存筛选结果: {output_path} ({len(filtered)} 条)")


# =====================================================
# 主逻辑
# =====================================================
def main():
    print(f"=== person+score+patch包含筛选开始 (VERSION={VERSION}, SCORE≥{SCORE_THRESHOLD}) ===")

    patch_mapping = load_patch_mapping(ANNOT_PATH)
    print(f"✅ 已加载 patch 区域映射: {len(patch_mapping)} 张图片")

    for model_name, model_dir in MODEL_DIRS.items():
        input_path = os.path.join(model_dir, "res-std.json")
        output_path = os.path.join(model_dir, "res-std-filter.json")

        if not os.path.exists(input_path):
            print(f"⚠️ {model_name}: 未找到 res-std.json，跳过。")
            continue

        print(f"\n--- {model_name.upper()} ---")
        filtered, grouped = filter_json(input_path, patch_mapping, SCORE_THRESHOLD)

        if filtered is None:
            continue

        save_filtered(filtered, output_path)

        total = len(filtered)
        img_count = len(grouped)
        avg_boxes = total / img_count if img_count else 0
        print(f"📊 图片数: {img_count} | 保留检测框数: {total} | 平均每图检测框: {avg_boxes:.2f}")

    print(f"\n✅ 所有模型筛选完成 (person+score≥{SCORE_THRESHOLD}+包含patch)，输出为 res-std-filter-patchinside.json。")


if __name__ == "__main__":
    main()


=== person+score+patch包含筛选开始 (VERSION=0.5, SCORE≥0.7) ===
✅ 已加载 patch 区域映射: 288 张图片

--- DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/detr/patch/res-std-filter.json (586 条)
📊 图片数: 287 | 保留检测框数: 586 | 平均每图检测框: 2.04

--- DEFORMABLE-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/deformable-detr/patch/res-std-filter.json (386 条)
📊 图片数: 233 | 保留检测框数: 386 | 平均每图检测框: 1.66

--- SPARSE-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/sparse-detr/patch/res-std-filter.json (496 条)
📊 图片数: 269 | 保留检测框数: 496 | 平均每图检测框: 1.84

--- ANCHOR-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/anchor-detr/patch/res-std-filter.json (414 条)
📊 图片数: 236 | 保留检测框数: 414 | 平均每图检测框: 1.75

--- DN-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/dn-detr/patch/res-std-filter.json (433 条)
📊 图片数: 232 | 保留检测框数: 433 | 平均每图检测框: 1.87

✅ 所有模型筛选完成 (person+score≥0.7+包含patch)，输出为 res-std-filter-patchinside.

In [3]:
# 统一名称；筛选；保存log

In [16]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm
from collections import defaultdict

# =====================================================
# ✅ 版本控制变量 —— 只需修改这一行
# =====================================================
VERSION = "0.5"
SCORE_THRESHOLD = 0.7  # 可灵活修改置信度阈值

# =====================================================
# 路径与模型配置
# =====================================================
BASE_DIR = "/opt/data/private/BlackBox"
SAVE_DIR = f"{BASE_DIR}/save-{VERSION}/attack/detection"

MODEL_DIRS = {
    "detr": f"{SAVE_DIR}/detr/patch/",
    "deformable-detr": f"{SAVE_DIR}/deformable-detr/patch/",
    "sparse-detr": f"{SAVE_DIR}/sparse-detr/patch/",
    "anchor-detr": f"{SAVE_DIR}/anchor-detr/patch/",
    "dn-detr": f"{SAVE_DIR}/dn-detr/patch/",
}

# =====================================================
# 核心函数
# =====================================================
def filter_json(input_path, score_thr=0.7):
    """读取res-std.json并筛选person+score>=thr"""
    if not os.path.exists(input_path):
        print(f"⚠️ 文件不存在: {input_path}")
        return None, None

    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if not isinstance(data, list):
        print(f"⚠️ 文件格式异常(非list): {input_path}")
        return None, None

    filtered = [
        det for det in data
        if det.get("category_id") == 1 and det.get("score", 0) >= score_thr
    ]

    grouped = defaultdict(list)
    for det in filtered:
        grouped[det["image_id"]].append(det)

    return filtered, grouped


def save_filtered(filtered, output_path):
    """保存筛选结果"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(filtered, f, indent=2, ensure_ascii=False)
    print(f"✅ 已保存筛选结果: {output_path} ({len(filtered)} 条)")


# =====================================================
# 主逻辑
# =====================================================
def main():
    print(f"=== 统一person筛选开始 (VERSION={VERSION}, SCORE≥{SCORE_THRESHOLD}) ===")

    for model_name, model_dir in MODEL_DIRS.items():
        input_path = os.path.join(model_dir, "res-std.json")
        output_path = os.path.join(model_dir, "res-std-filter.json")

        if not os.path.exists(input_path):
            print(f"⚠️ {model_name}: 未找到 res-std.json，跳过。")
            continue

        print(f"\n--- {model_name.upper()} ---")
        filtered, grouped = filter_json(input_path, SCORE_THRESHOLD)

        if filtered is None:
            continue

        save_filtered(filtered, output_path)

        # 控制台统计信息
        total = len(filtered)
        img_count = len(grouped)
        avg_boxes = total / img_count if img_count else 0
        print(f"📊 图片数: {img_count} | person框数: {total} | 平均每图检测框: {avg_boxes:.2f}")

    print(f"\n✅ 所有模型person筛选完成 (score≥{SCORE_THRESHOLD})，输出为 res-std-filter.json。")


if __name__ == "__main__":
    main()


=== 统一person筛选开始 (VERSION=0.5, SCORE≥0.7) ===

--- DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/detr/patch/res-std-filter.json (1163 条)
📊 图片数: 288 | person框数: 1163 | 平均每图检测框: 4.04

--- DEFORMABLE-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/deformable-detr/patch/res-std-filter.json (557 条)
📊 图片数: 271 | person框数: 557 | 平均每图检测框: 2.06

--- SPARSE-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/sparse-detr/patch/res-std-filter.json (646 条)
📊 图片数: 279 | person框数: 646 | 平均每图检测框: 2.32

--- ANCHOR-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/anchor-detr/patch/res-std-filter.json (608 条)
📊 图片数: 271 | person框数: 608 | 平均每图检测框: 2.24

--- DN-DETR ---
✅ 已保存筛选结果: /opt/data/private/BlackBox/save-0.5/attack/detection/dn-detr/patch/res-std-filter.json (582 条)
📊 图片数: 277 | person框数: 582 | 平均每图检测框: 2.10

✅ 所有模型person筛选完成 (score≥0.7)，输出为 res-std-filter.json。


In [13]:
# 筛选patched目标检测框 标注的人物框内（IoU ≥ 0.5）

In [15]:
import os
import json
from collections import defaultdict
from tqdm import tqdm

# =====================================================
# ✅ 版本号与阈值设置
# =====================================================
VERSION = "tt"
IOU_THRESHOLD = 0.5  # IoU阈值

# =====================================================
# 路径配置
# =====================================================
BASE_DIR = "/opt/data/private/BlackBox"
SAVE_DIR = f"{BASE_DIR}/save-{VERSION}/attack/detection"
ANNOTATION_PATH = f"{BASE_DIR}/data/coco/annotations/instances_val2017.json"

MODEL_DIRS = {
    "detr": f"{SAVE_DIR}/detr/patch/",
    "deformable-detr": f"{SAVE_DIR}/deformable-detr/patch/",
    "sparse-detr": f"{SAVE_DIR}/sparse-detr/patch/",
    "anchor-detr": f"{SAVE_DIR}/anchor-detr/patch/",
    "dn-detr": f"{SAVE_DIR}/dn-detr/patch/",
}

# =====================================================
# 工具函数
# =====================================================
def xywh_to_xyxy(bbox):
    """将 [x,y,w,h] 转换为 [x1,y1,x2,y2]"""
    if len(bbox) != 4:
        return bbox
    x, y, w, h = bbox
    return [x, y, x + w, y + h]

def compute_iou(box1, box2):
    """计算两个bbox的IoU（输入为[x1,y1,x2,y2]）"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_w = max(0, x2 - x1)
    inter_h = max(0, y2 - y1)
    inter_area = inter_w * inter_h
    if inter_area == 0:
        return 0.0
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter_area
    return inter_area / union if union > 0 else 0.0

def load_gt_boxes(annotation_path):
    """加载原始COCO标注中的person类别GT框"""
    with open(annotation_path, "r") as f:
        coco_data = json.load(f)

    gt_boxes = defaultdict(list)
    for ann in coco_data.get("annotations", []):
        if ann.get("category_id") == 1:  # person
            box = ann.get("bbox", [])
            if len(box) == 4:
                gt_boxes[ann["image_id"]].append(xywh_to_xyxy(box))
    return gt_boxes

# =====================================================
# 主过滤逻辑
# =====================================================
def iou_filter(input_json_path, gt_boxes, iou_threshold=0.5):
    """对 result-filter.json 执行 IoU 筛选，输出 result-patch-filter.json"""
    if not os.path.exists(input_json_path):
        print(f"⚠️ 文件不存在：{input_json_path}")
        return None

    # 加载预测结果
    with open(input_json_path, "r") as f:
        preds = json.load(f)

    print(f"\n🧩 加载检测结果: {len(preds)} 个候选框")
    matched = []
    for det in preds:
        img_id = det.get("image_id")
        box_pred = det.get("bbox", [])
        if len(box_pred) != 4:
            continue
        gt_list = gt_boxes.get(img_id, [])
        if any(compute_iou(box_pred, gt) >= iou_threshold for gt in gt_list):
            matched.append(det)

    # 保存新结果
    output_path = os.path.join(os.path.dirname(input_json_path), "result-patch-filter.json")
    with open(output_path, "w") as f:
        json.dump(matched, f, indent=2)
    print(f"✅ 筛选后剩余 {len(matched)} 个框（IoU≥{iou_threshold}），结果已保存: {output_path}")
    return output_path

# =====================================================
# 主程序
# =====================================================
def main():
    print(f"=== 基于 GT 的二次过滤 (VERSION={VERSION}, IoU≥{IOU_THRESHOLD}) ===")

    gt_boxes = load_gt_boxes(ANNOTATION_PATH)
    print(f"📦 已加载 GT person 标注，共 {sum(len(v) for v in gt_boxes.values())} 个框。")

    for model_name, model_path in tqdm(MODEL_DIRS.items()):
        input_json = os.path.join(model_path, "result-filter.json")
        if not os.path.exists(input_json):
            print(f"⚠️ {model_name}: 未找到 result-filter.json，跳过。")
            continue
        print(f"\n=== {model_name.upper()} 模型 IoU 过滤 ===")
        iou_filter(input_json, gt_boxes, IOU_THRESHOLD)

    print(f"\n✅ 所有模型已生成 result-patch-filter.json 文件。")

if __name__ == "__main__":
    main()


=== 基于 GT 的二次过滤 (VERSION=tt, IoU≥0.5) ===
📦 已加载 GT person 标注，共 597 个框。


100%|██████████| 5/5 [00:00<00:00, 63.99it/s]


=== DETR 模型 IoU 过滤 ===

🧩 加载检测结果: 1157 个候选框
✅ 筛选后剩余 588 个框（IoU≥0.5），结果已保存: /opt/data/private/BlackBox/save-tt/attack/detection/detr/patch/result-patch-filter.json

=== DEFORMABLE-DETR 模型 IoU 过滤 ===

🧩 加载检测结果: 480 个候选框
✅ 筛选后剩余 417 个框（IoU≥0.5），结果已保存: /opt/data/private/BlackBox/save-tt/attack/detection/deformable-detr/patch/result-patch-filter.json

=== SPARSE-DETR 模型 IoU 过滤 ===

🧩 加载检测结果: 626 个候选框
✅ 筛选后剩余 477 个框（IoU≥0.5），结果已保存: /opt/data/private/BlackBox/save-tt/attack/detection/sparse-detr/patch/result-patch-filter.json

=== ANCHOR-DETR 模型 IoU 过滤 ===

🧩 加载检测结果: 581 个候选框
✅ 筛选后剩余 445 个框（IoU≥0.5），结果已保存: /opt/data/private/BlackBox/save-tt/attack/detection/anchor-detr/patch/result-patch-filter.json

=== DN-DETR 模型 IoU 过滤 ===

🧩 加载检测结果: 565 个候选框
✅ 筛选后剩余 457 个框（IoU≥0.5），结果已保存: /opt/data/private/BlackBox/save-tt/attack/detection/dn-detr/patch/result-patch-filter.json

✅ 所有模型已生成 result-patch-filter.json 文件。
